In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float

In [2]:
covid_file = "../Resources/1_county_level_confirmed_cases.csv"
covid_df = pd.read_csv(covid_file)
covid_df.head()

,last_update,location_type,state,county_name,county_name_long,fips_code,nchs_urbanization,total_population,confirmed,confirmed_per_100000,deaths,deaths_per_100000
0,2020-09-04 00:28:22 UTC,county,Alabama,Autauga,"Autauga, Alabama, US",1001,Medium metro,55200.0,1349,2444.0,23,42.0
1,2020-09-04 00:28:22 UTC,county,Alabama,Baldwin,"Baldwin, Alabama, US",1003,Small metro,208107.0,4495,2160.0,40,19.0
2,2020-09-04 00:28:22 UTC,county,Alabama,Barbour,"Barbour, Alabama, US",1005,Non-core,25782.0,614,2382.0,7,27.0
3,2020-09-04 00:28:22 UTC,county,Alabama,Bibb,"Bibb, Alabama, US",1007,Large fringe metro,22527.0,542,2406.0,6,27.0
4,2020-09-04 00:28:22 UTC,county,Alabama,Blount,"Blount, Alabama, US",1009,Large fringe metro,57645.0,1037,1799.0,11,19.0


### Extract CSVs into DataFrames

In [3]:
unemployment_file = "../Resources/Unemployment.csv"
unemployment_df = pd.read_csv(unemployment_file)
unemployment_df.head()

,fips_code,Stabr,area_name,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2018
0,0,US,United States,"157,115,247","5,984,808",3.7,"61,937"
1,1000,AL,Alabama,"2,174,483","67,264",3.0,"49,881"
2,1001,AL,"Autauga County, AL","25,458",714,2.7,"59,338"
3,1003,AL,"Baldwin County, AL","94,675","2,653",2.7,"57,588"
4,1005,AL,"Barbour County, AL","8,213",324,3.8,"34,382"


### Transform Covid DataFrame

In [4]:
covid_df1 = covid_df.drop(columns=['nchs_urbanization'])
covid_df2 = covid_df1.dropna()
covid_df2.head()
 
# Create a filtered dataframe from specific columns
#premise_cols = ["License Serial Number", "Premises Name", "County ID Code"]
#premise_transformed= premise_df[premise_cols].copy()

# Rename the column headers
#premise_transformed = premise_transformed.rename(columns={"License Serial Number": "id",

# Clean the data by dropping duplicates and setting the index
#premise_transformed.drop_duplicates("id", inplace=True)
#premise_transformed.set_index("id", inplace=True)

#premise_transformed.head()

,last_update,location_type,state,county_name,county_name_long,fips_code,total_population,confirmed,confirmed_per_100000,deaths,deaths_per_100000
0,2020-09-04 00:28:22 UTC,county,Alabama,Autauga,"Autauga, Alabama, US",1001,55200.0,1349,2444.0,23,42.0
1,2020-09-04 00:28:22 UTC,county,Alabama,Baldwin,"Baldwin, Alabama, US",1003,208107.0,4495,2160.0,40,19.0
2,2020-09-04 00:28:22 UTC,county,Alabama,Barbour,"Barbour, Alabama, US",1005,25782.0,614,2382.0,7,27.0
3,2020-09-04 00:28:22 UTC,county,Alabama,Bibb,"Bibb, Alabama, US",1007,22527.0,542,2406.0,6,27.0
4,2020-09-04 00:28:22 UTC,county,Alabama,Blount,"Blount, Alabama, US",1009,57645.0,1037,1799.0,11,19.0


### Transform county DataFrame

In [5]:
unemployment_df1 = unemployment_df.drop(columns=['area_name','Stabr'])
unemployment_df1.head()

,fips_code,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2018
0,0,"157,115,247","5,984,808",3.7,"61,937"
1,1000,"2,174,483","67,264",3.0,"49,881"
2,1001,"25,458",714,2.7,"59,338"
3,1003,"94,675","2,653",2.7,"57,588"
4,1005,"8,213",324,3.8,"34,382"


### Create tables and database connection

In [6]:
Base = declarative_base()


# Creates Classes which will serve as the anchor points for our Tables
class Covid(Base):
    __tablename__ = 'covid'
    id = Column(Integer, primary_key=True)
    last_update = Column(String(255))
    location_type = Column(String(255))
    state = Column(String(255))
    county_name = Column(String(255))
    county_name_long = Column(String(255))
    fips_code = Column(Integer)
    total_population = Column(Integer)
    confirmed = Column(Integer)
    confirmed_per_100000 = Column(Integer)
    deaths = Column(Integer)
    deaths_per_100000 = Column(Integer)
    

class County(Base):
    __tablename__ = 'county'
    id = Column(Integer, primary_key=True)
    fips_code = Column(Integer)
    Employed_2019 = Column(Integer)
    Unemployment_rate_2019 = Column(Integer)
    Median_Household_Income_2018 = Column(Integer)


In [7]:
engine = create_engine('sqlite:///covid.db',echo=True)
connection = engine.connect()
Base.metadata.create_all(engine)

2020-09-07 16:06:25,820 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-09-07 16:06:25,821 INFO sqlalchemy.engine.base.Engine ()
2020-09-07 16:06:25,823 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-09-07 16:06:25,824 INFO sqlalchemy.engine.base.Engine ()
2020-09-07 16:06:25,826 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("covid")
2020-09-07 16:06:25,827 INFO sqlalchemy.engine.base.Engine ()
2020-09-07 16:06:25,829 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("covid")
2020-09-07 16:06:25,830 INFO sqlalchemy.engine.base.Engine ()
2020-09-07 16:06:25,831 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("county")
2020-09-07 16:06:25,833 INFO sqlalchemy.engine.base.Engine ()
2020-09-07 16:06:25,834 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("county")
2020-09-07 16:06:25,835 INFO sqlalchemy.engine.base.Engine ()
2020-09-07 16:06:25,837 

In [8]:
# Confirm tables
engine.table_names()

2020-09-07 16:06:27,690 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-09-07 16:06:27,691 INFO sqlalchemy.engine.base.Engine ()


['county', 'covid']

### Load DataFrames into database

In [9]:
covid_df2.to_sql(name='covid', con=engine, if_exists='replace', index=True)

2020-09-07 16:06:29,627 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("covid")
2020-09-07 16:06:29,628 INFO sqlalchemy.engine.base.Engine ()
2020-09-07 16:06:29,631 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("covid")
2020-09-07 16:06:29,632 INFO sqlalchemy.engine.base.Engine ()
2020-09-07 16:06:29,634 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-09-07 16:06:29,634 INFO sqlalchemy.engine.base.Engine ()
2020-09-07 16:06:29,636 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_xinfo("covid")
2020-09-07 16:06:29,637 INFO sqlalchemy.engine.base.Engine ()
2020-09-07 16:06:29,639 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'covid' AND type = 'table'
2020-09-07 16:06:29,640 INFO sqlalchemy.engine.base.Engine ()
2020-09-07 16:06:29,641 INFO sqlalchemy.engine.base.Engine PRAGMA main.foreign_key_list("covid")
2020-

In [10]:
unemployment_df1.to_sql(name='county', con=engine, if_exists='replace', index=True)

2020-09-07 16:06:33,543 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("county")
2020-09-07 16:06:33,544 INFO sqlalchemy.engine.base.Engine ()
2020-09-07 16:06:33,546 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("county")
2020-09-07 16:06:33,546 INFO sqlalchemy.engine.base.Engine ()
2020-09-07 16:06:33,548 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-09-07 16:06:33,549 INFO sqlalchemy.engine.base.Engine ()
2020-09-07 16:06:33,551 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_xinfo("county")
2020-09-07 16:06:33,551 INFO sqlalchemy.engine.base.Engine ()
2020-09-07 16:06:33,553 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'county' AND type = 'table'
2020-09-07 16:06:33,554 INFO sqlalchemy.engine.base.Engine ()
2020-09-07 16:06:33,555 INFO sqlalchemy.engine.base.Engine PRAGMA main.foreign_key_list("county")


In [ ]:
connection.close()

In [ ]:
engine = db.create_engine('sqlite:///customer_db.db',echo=True)
connection = engine.connect()

In [ ]:
engine.execute('select * from premise')

In [ ]:
with engine.connect() as con:

    rs = con.execute('SELECT * FROM premise')

    for row in rs:
        print(row)

In [ ]:
rs = engine.execute('SELECT premise.id, premise.premise_name, county.county_name\
FROM premise\
INNER JOIN county\
ON premise.county_id = county.county_id;')

In [ ]:
print(rs)

In [ ]:
for i in rs:
    print(i)